In [1]:
import os
import numpy as np
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4041 --packages org.apache.kafka:kafka_2.11:0.9.0.0,org.apache.kafka:kafka-clients:0.9.0.0  pyspark-shell'

In [2]:
from pyspark import SparkContext
sc = SparkContext("local[1]", "KafkaSendStream") 
from kafka import KafkaProducer
import time

In [ ]:
import json
from satori.rtm.client import make_client, SubscriptionMode
from math import log

# Make a proxy class

endpoint = "wss://open-data.api.satori.com"
appkey = "dC6c33Fbb5ECdAC1Ef2aB77dcBfBB0B0"
channel = "cryptocurrency-market-data"

with make_client(endpoint=endpoint, appkey=appkey) as client:
    print('Connected to Satori RTM!')
    messagebox=[]

    class SubscriptionObserver(object):
        def on_subscription_data(self, data):
            for message in data['messages']:
                messagebox.append(message)

    subscription_observer = SubscriptionObserver()
    client.subscribe(
        channel,
        SubscriptionMode.SIMPLE,
        subscription_observer,
        {'filter':'select * from `cryptocurrency-market-data` where exchange = "Bitstamp" and basecurrency = "USD" and cryptocurrency="BTC"'})
        #  exchange != "ShapheShift" OU type != "ratequote" si on laisse ShapeShift

    producer = KafkaProducer(bootstrap_servers='localhost:9092')
    i=0
    numOfColumnsX=3   # number of columns of X counting t_0 column with only ones
    numOfRendements=10
    timeBetweenPrices=3
    sumOfProxies=np.zeros(numOfColumnsX)
    sumOfSquaredProxies=np.zeros(numOfColumnsX)
    while not messagebox: # waiting for a first price to arrive
        time.sleep(1)
    firstMessage=messagebox.pop()
    firstMessage=json.dumps(firstMessage, ensure_ascii=False)
    firstMessage=json.loads(firstMessage)
    lastPrice=float(firstMessage['price']) # get that first price
    lastTime=int(firstMessage['timestamp'])
    rendements=np.empty(0)
    proxies=np.empty(0)
    interpolatedTimes=[]
    interpolatedPrices=[]
    while True:
        proxies=proxies[1:]
        while proxies.size<numOfColumnsX: # Pas -1 car le dernier proxy sera utilisé comme y            
            if messagebox and not interpolatedTimes:
                newMessage=messagebox.pop()
                newMessage=json.dumps(newMessage, ensure_ascii=False)
                newMessage=json.loads(newMessage)
                prices=[lastPrice, float(newMessage['price'])]
                times=[lastTime, int(newMessage['timestamp'])]
                interpolatedTimes=list(range(lastTime, int(newMessage['timestamp']), timeBetweenPrices))
                interpolatedPrices=list(np.interp(interpolatedTimes, times, prices))
            while interpolatedTimes and proxies.size<numOfColumnsX:
                newPrice=interpolatedPrices.pop(0)
                rendement=np.log(newPrice/lastPrice)
                lastPrice=newPrice
                lastTime=interpolatedTimes.pop(0)
                rendements=np.append(rendements, rendement)
                if rendements.size==numOfRendements:
                    proxies=np.append(proxies,np.sum(np.square(rendements))/numOfRendements)
                    rendements=rendements[1:]
            time.sleep(1)
        
        sumOfProxies+=proxies
        sumOfSquaredProxies+=proxies**2
        mean=sumOfProxies/(i+1)
        std=np.sqrt(sumOfSquaredProxies/(i+1)-mean**2) # https://math.stackexchange.com/a/1379804
        print(mean,std)
        if i!=0:
            y=(proxies[-1]-mean[-1])/std[-1]
            message=np.array2string(np.append([i,y], np.insert((proxies[:numOfColumnsX-1]-mean[:numOfColumnsX-1])/std[:numOfColumnsX-1],0,1)),separator=",") # Ajout de la colonne t_0 aux proxies 
            producer.send('volatility', message)
            print(message)
        i=i+1

Connected to Satori RTM!
(array([  1.21474557e-11,   1.21582250e-11,   9.73524313e-12]), array([ 0.,  0.,  0.]))
(array([  1.21528403e-11,   1.09467341e-11,   3.71378396e-10]), array([  5.38465502e-15,   1.21149092e-12,   3.61643153e-10]))
[ 1., 1., 1., 1.,-1.]
(array([  1.13469746e-11,   2.51638339e-10,   7.33062761e-10]), array([  1.13967473e-12,   3.40390769e-10,   5.90611271e-10]))
[ 2.        , 1.22477976, 1.        ,-1.41420304, 1.41420759]
(array([  1.91765618e-10,   5.52836627e-10,   1.09478835e-09]), array([  3.12495816e-10,   5.99216683e-10,   8.08796916e-10]))
[ 3.        , 1.34171723, 1.        , 1.73204217, 1.50796013]
(array([  4.44698793e-10,   8.78262322e-10,   1.45655516e-09]), array([  5.77947810e-10,   8.43122789e-10,   1.02314362e-09]))
[ 4.        , 1.41433443, 1.        , 1.75056066, 1.54390653]
(array([  7.33909844e-10,   1.21582234e-09,   1.81876706e-09]), array([  8.34606626e-10,   1.07801384e-09,   1.23626036e-09]))
[ 5.        , 1.4649499 , 1.        , 1.7326

(array([  4.93088926e-09,   5.03507551e-09,   5.17137270e-09]), array([  3.67987164e-09,   3.60473784e-09,   3.52736453e-09]))
[ 44.        ,  0.27617197,  1.        , -0.4555024 , -0.09280764]
(array([  4.92588142e-09,   5.05921586e-09,   5.22394525e-09]), array([  3.63980829e-09,   3.56901636e-09,   3.50659236e-09]))
[ 45.        ,  0.67466204,  1.        , -0.06191346,  0.30437407]
(array([  4.95183143e-09,   5.11305616e-09,   5.30499030e-09]), array([  3.60517742e-09,   3.54967644e-09,   3.51236541e-09]))
[ 46.        ,  1.06141364,  1.        ,  0.33110728,  0.69771256]
(array([  5.00678723e-09,   5.19472297e-09,   5.42967022e-09]), array([  3.58726549e-09,   3.55684742e-09,   3.57915001e-09]))
[ 47.        ,  1.63724795,  1.        ,  0.72002554,  1.079141  ]
(array([  5.08895612e-09,   5.31910875e-09,   5.59529951e-09]), array([  3.59582183e-09,   3.62431015e-09,   3.72366309e-09]))
[ 48.        ,  2.13504976,  1.        ,  1.09685822,  1.64735281]
(array([  5.21296952e-09,   5.

(array([  4.25365528e-09,   4.27872211e-09,   4.30381498e-09]), array([  4.34136981e-09,   4.32313724e-09,   4.30466894e-09]))
[ 87.        , -0.48400565,  1.        , -0.46942104, -0.47666637]
(array([  4.23078307e-09,   4.25559412e-09,   4.28043092e-09]), array([  4.32223998e-09,   4.30425285e-09,   4.28603431e-09]))
[ 88.        , -0.48011685,  1.        , -0.46567407, -0.47284937]
(array([  4.20844471e-09,   4.23300567e-09,   4.25763414e-09]), array([  4.30332366e-09,   4.28557496e-09,   4.26757902e-09]))
[ 89.        , -0.4754249 ,  1.        , -0.46199496, -0.46910206]
(array([  4.18662261e-09,   4.21098056e-09,   4.22936470e-09]), array([  4.28461805e-09,   4.26708172e-09,   4.25253111e-09]))
[ 90.        , -0.59829074,  1.        , -0.45838135, -0.46454694]
(array([  4.16534107e-09,   4.18352549e-09,   4.19573576e-09]), array([  4.26610163e-09,   4.25190164e-09,   4.24150536e-09]))
[ 91.        , -0.72149685,  1.        , -0.45395549, -0.58759844]
(array([  4.13867197e-09,   4.

(array([  3.28889990e-09,   3.32173241e-09,   3.35247802e-09]), array([  3.89893422e-09,   3.88916169e-09,   3.87866107e-09]))
[ 129.        ,   0.16928791,   1.        ,   0.32386569,   0.24649057]
(array([  3.29646840e-09,   3.32697940e-09,   3.35541922e-09]), array([  3.88498280e-09,   3.87475099e-09,   3.86397418e-09]))
[  1.30000000e+02,  9.89537569e-02,  1.00000000e+00,  2.53258695e-01,
   1.76039451e-01]
(array([  3.30186704e-09,   3.33009148e-09,   3.35834093e-09]), array([  3.87073220e-09,   3.86021034e-09,   3.84945534e-09]))
[  1.31000000e+02,  9.94283811e-02,  1.00000000e+00,  1.82709979e-01,
   1.05611472e-01]
(array([  3.30514453e-09,   3.33318167e-09,   3.35561101e-09]), array([  3.85633697e-09,   3.84583477e-09,   3.83508466e-09]))
[  1.32000000e+02, -9.39614231e-02,  1.00000000e+00,  1.12186629e-01,
   1.06063938e-01]
(array([  3.30839783e-09,   3.33065987e-09,   3.34744954e-09]), array([  3.84210390e-09,   3.83156815e-09,   3.82190700e-09]))
[  1.33000000e+02, -2.8401

(array([  2.88824842e-09,   2.89254838e-09,   2.91143725e-09]), array([  3.55724205e-09,   3.55420336e-09,   3.54737146e-09]))
[  1.69000000e+02,  8.79041485e-02,  1.00000000e+00, -5.82823431e-01,
  -6.04750764e-01]
(array([  2.87570393e-09,   2.89448240e-09,   2.92775972e-09]), array([  3.55059475e-09,   3.54388543e-09,   3.54338065e-09]))
[  1.70000000e+02,  7.83099501e-01,  1.00000000e+00, -6.00621275e-01,
   9.27748717e-02]
(array([  2.87772464e-09,   2.91080855e-09,   2.95830229e-09]), array([  3.54035682e-09,   3.54001195e-09,   3.55556834e-09]))
[  1.71000000e+02,  1.46890147e+00,  1.00000000e+00,  9.76006924e-02,
   7.88633155e-01]
(array([  2.89405328e-09,   2.94127255e-09,   3.00281368e-09]), array([  3.53659925e-09,   3.55230527e-09,   3.59301658e-09]))
[ 172.        ,   2.13078898,   1.        ,   0.79413184,   1.47504477]
(array([  2.92443850e-09,   2.98562601e-09,   3.06146130e-09]), array([  3.54899650e-09,   3.58980237e-09,   3.66478036e-09]))
[ 173.        ,   2.768525

(array([  3.11131526e-09,   3.12331609e-09,   3.13772723e-09]), array([  4.08258065e-09,   4.07717105e-09,   4.07151890e-09]))
[  2.10000000e+02, -2.08323731e-02,  1.00000000e+00, -2.63481299e-01,
  -1.42008452e-01]
(array([  3.10864077e-09,   3.12298398e-09,   3.13732697e-09]), array([  4.07312582e-09,   4.06754658e-09,   4.06190907e-09]))
[  2.11000000e+02, -2.07916632e-02,  1.00000000e+00, -1.38546524e-01,
  -1.72281126e-02]
(array([  3.10837911e-09,   3.12265482e-09,   3.13697276e-09]), array([  4.06355503e-09,   4.05798994e-09,   4.05236614e-09]))
[  2.12000000e+02, -1.85306417e-02,  1.00000000e+00, -1.36509249e-02,
  -1.71961223e-02]
(array([  3.10811974e-09,   3.12237082e-09,   3.13666395e-09]), array([  4.05405139e-09,   4.04849968e-09,   4.04288942e-09]))
[  2.13000000e+02, -1.62699511e-02,  1.00000000e+00, -1.36275062e-02,
  -1.49415864e-02]
(array([  3.10790467e-09,   3.12213136e-09,   3.13640006e-09]), array([  4.04461360e-09,   4.03907511e-09,   4.03347824e-09]))
[  2.1400

(array([  2.80297332e-09,   2.82091020e-09,   2.84168990e-09]), array([  3.84226481e-09,   3.83970239e-09,   3.83855876e-09]))
[ 250.        ,   0.6216325 ,   1.        ,   0.25997674,   0.44102257]
(array([  2.80976432e-09,   2.83046160e-09,   2.85194765e-09]), array([  3.83614274e-09,   3.83506294e-09,   3.83438047e-09]))
[ 251.        ,   0.67147631,   1.        ,   0.4443373 ,   0.62512695]
(array([  2.81932202e-09,   2.84072319e-09,   2.86009039e-09]), array([  3.83155912e-09,   3.83094116e-09,   3.82897763e-09]))
[ 252.        ,   0.53590562,   1.        ,   0.62860593,   0.67500909]
(array([  2.82958706e-09,   2.84887806e-09,   2.88028696e-09]), array([  3.82749338e-09,   3.82559214e-09,   3.83491174e-09]))
[ 253.        ,   1.33242514,   1.        ,   0.67852664,   0.53931075]
(array([  2.83775363e-09,   2.86903940e-09,   2.91239051e-09]), array([  3.82219776e-09,   3.83158039e-09,   3.86143202e-09]))
[ 254.        ,   2.11172962,   1.        ,   0.54270017,   1.3365191 ]
(arra